In [12]:
import json
import re
import numpy as np

In [13]:
WAKEWORDS = ["ALEXA", "AMAZON", "ECHO", "COMPUTER"]
SRAI_PHRASE = "CAN YOU DO ALEXA THINGS"

ANSWERS = [
    "Sorry, I am a socialbot, I cannot do this.",
    "Unfortunately, I cannot do this because I am a socialbot.",
    "I am really sorry but I am a socialbot, and I cannot do some Alexa things."
]

ADD_WORDS = [
    "PLEASE ",
    "COULD YOU",
    "CAN YOU"
]

In [14]:
VERB_COMMANDS = [
    # SIGN `#` MEANS 0 OR MORE SYMBOLS
    # SIGN `*` MEANS 1 OR MORE SYMBOLS REQUIRED
    #
    #COMMANDS
    "MUTE #",
    "UNMUTE #",
    "RESUME #",
    "SHOW #",
    "PLAY #",
    "READ #",
    "RESTART #",
    "OPEN #",
    "START #",
    "LAUNCH #",
    "RETURN #",
    "SET #",
    "WATCH #",
    "PAUSE #",
    "REWIND #",
    "FAST FORWARD #",
    "TURN # UP #",
    "TURN # DOWN #",
    "LOUDER #",
    "QUIETER #",
    "DECREASE #",
    "INCREASE #",
    "THUMBS UP",
    "THUMBS DOWN",
    "BASSES UP",
    "BASSES DOWN",
    "CREATE #",
    "NEED #",
    "MOVE #",
    "MAKE # DONATION #",
    "ENABLE #",
    "DISABLE #",
    "GIVE #",
    "SAMPLE #",
    "DIM # LIGHTS #",
    "DIM # LIGHT #",
    "RAISE #",
    "MAKE # LIGHTS #",
    "MAKE # LIGHT #",
    "LOCK # DOOR #",
    "UNLOCK # DOOR #",
    "CLOSE # DOOR",
    "LOCK # DOORS #",
    "UNLOCK # DOORS",
    "CLOSE # DOORS",
    "DISCOVER # DIVICES #",
    "TRIGGER #",
    "CONNECT # TO #",
    "DISCONNECT # FROM #",
    "SWITCH # ACCOUNT #",
    "SWITCH # ACCOUNTS #",
    "TRACK #",
    "SING #",
    # IF ABOUT STOP PLAYING, STOP MUSIC ETC
    "STOP *",  
    # BECAUSE WE CAN DO SIMPLE SEARCH FOR INFO ABOUT SMTH
    "SEARCH FOR # ON FIRE TV",  
    "SEARCH FOR # IN MY #", 
    "SEARCH FOR # ON MY #", 
    "TURN # ON #",
    "TURN # OFF #",
    "GO TO #",
    "SWITCH TO #",
    "STOP IN #",
    "NEXT IN #",
    "ADD # SONG #", #  `add this song` adds the song to prime music lib
    "ADD # BOOK #",
    "ADD # TO #",
    "CREATE # PLAYLIST #",
    "ADD # PLAYLIST #",
    # REMINDERS
    "REMIND #",
    "WAKE # UP #",
    "SNOOZE #",
    "# ALARM #",
    "# REMINDER #",
    "# TIMER #",
    "# ALARMS #",
    "# REMINDERS #",
    "# TIMERS #",
    # CALLS AND MESSAGES
    "CALL #",
    "ANSWER # CALL #",
    "ANSWER",
    "HANG UP #",
    "END # CALL #",
    "SEND #",
    "DROP IN #",  # lets users call different Echo devices throughout their home, intercom style. 
    "ANNOUNCE #",
    "BROADCAST #",
    "TELL # EVERYONE #",
    "# MESSAGE #",
    "# MESSAGES #",
    "# INFORMATION #",
    # SHOPPING
    "BUY #",
    "REORDER #",
    "ORDER #",
    "ADD # TO # CART #",
    "REQUEST #",
    "ASK # TO #",
    "ASK # FOR #",
    "SHOP # FOR",
    "# DEAL #",
    "# DEALS #",
    "FIND # IN #",
    "FIND # ON #",
    "FIND # NEARBY #",
    "FIND # CLOSEST #",
    "FIND # NEAREST #",
    "QUOTE #",
#     "",
#     "",
#     "",
#     "",
#     "",
#     "",
#     "",
#     "",
#     "",
]
PHRASE_COMMANDS = [
    # SIMPLE QUESTIONS ABOUT PLANS FROM CALENDAR AND REMINDERS
    # 
    # WE CAN ANSWER THIS QUESTIONS IF WE ALREADY KNOW TIME AND DATE REGION OF ECHO
    # "WHAT TIME IS IT #",
    # "WHAT DATE IS IT #",
    # "WHAT IS THE TIME #",
    # "WHAT IS THE DATE #",
    "WHEN # MEETING #", # WE SHOULD ANSWER QUESTIONS LIKE `WHEN THE EASTER THIS YEAR?`
    "WHEN # CALL #",
    "WHEN # MY #",
    "WHEN # ME #",
    "WHEN # OUR #",
    "WHEN # I #",
    "WHEN # TODAY #",  # THIS IS MOSTLY ABOUT CALENDAR
    "WHEN # THIS DAY #",  # THIS IS MOSTLY ABOUT CALENDAR
    # REGULAR SPECIAL PHRASES TO ALEXA
    "WHERE IS MY STUFF?",
    "WHAT CAN YOU SHOW ME",
    "WHAT ARE YOUR DEALS",
    "WHAT # MY #",
    "WHAT # ME #",
    "HELP",
    "# NOTIFICATIONS #",
    "WHAT DID I MISS",
    "RUN AWAY MODE",
    "I AM LEAVING",
    "I LIKE THIS *",
    "BLUETOOTH",
    "LIGHTS OFF #",
    "LIGHTS ON #",
    "LIGHTS OUT #",
    "BEDTIME #",
    # TALK ABOUT MOVIES
    "WHAT * ARE PLAYING",
    "WHAT # RATING #",
    "# QUOTE #",
    "WHO # PLAYS #",
    "WHAT # MOVIE #",
    "WHO SINGS #",
    # cannot answer without location -> will be separate skill
    "WHAT # NEARBY #",
    "WHAT # CLOSEST #",
    "WHAT # NEAREST #",
    # oracle skill
    "ROLL # DIE",
    "FLIP # COIN",
    "PICK A NUMBER #",
    "HOW # SPELL",
    "SPELL #",
#     "",
#     "",
#     "",
#     "",
#     "",
#     "",
#     "",
#     "",
#     "what's in the news",
#     "what's the weather like",
#     "will it rain today",
#     "will I need an umbrella today",
#     "what's the weather going to be like this weekend",
#     "what's traffic like",
#     "",
#     "",
]

In [15]:
TEMPLATE = """\n  <category>
    <pattern>PHRASE</pattern>
    <template>
      <srai>SRAITEMPLATE</srai>
    </template>
  </category>"""

TEMPLATE_ADD_WORDS = """\n  <category>
    <pattern>ADDWORD *</pattern>
    <template>
      <srai><star/></srai>
    </template>
  </category>"""

TEMPLATE_NAME = """  <category>
    <pattern>$NAME *</pattern>
    <template>
      <srai><star/></srai>
    </template>
  </category>
"""


In [16]:
cannot_string = """<?xml version="1.0" encoding="UTF-8"?>
<aiml version="2.0">
"""

for wakeword in WAKEWORDS:
    cannot_string += "\n" + TEMPLATE_NAME.replace("NAME", wakeword)
    
for addword in ADD_WORDS:
    cannot_string += "\n" + TEMPLATE_ADD_WORDS.replace("ADDWORD", addword)
    
print(cannot_string)

<?xml version="1.0" encoding="UTF-8"?>
<aiml version="2.0">

  <category>
    <pattern>$ALEXA *</pattern>
    <template>
      <srai><star/></srai>
    </template>
  </category>

  <category>
    <pattern>$AMAZON *</pattern>
    <template>
      <srai><star/></srai>
    </template>
  </category>

  <category>
    <pattern>$ECHO *</pattern>
    <template>
      <srai><star/></srai>
    </template>
  </category>

  <category>
    <pattern>$COMPUTER *</pattern>
    <template>
      <srai><star/></srai>
    </template>
  </category>


  <category>
    <pattern>PLEASE  *</pattern>
    <template>
      <srai><star/></srai>
    </template>
  </category>

  <category>
    <pattern>COULD YOU *</pattern>
    <template>
      <srai><star/></srai>
    </template>
  </category>

  <category>
    <pattern>CAN YOU *</pattern>
    <template>
      <srai><star/></srai>
    </template>
  </category>


In [17]:
for phrase in PHRASE_COMMANDS + VERB_COMMANDS:
    s = TEMPLATE.replace("PHRASE", phrase)
    s = s.replace("SRAITEMPLATE", SRAI_PHRASE)
    cannot_string += s

answers = """  <category>
    <pattern>CAN YOU DO ALEXA THINGS</pattern>
    <template>
      <random>LISTOFANSWERS
      </random>
    </template>
  </category>
</aiml>
"""
ans_string = ""
for ans in ANSWERS:
    ans_string += "\n        <li>" + ans + "</li>"
answers = answers.replace("LISTOFANSWERS", ans_string)

cannot_string += "\n\n" + answers

In [18]:
with open("cannot.aiml", "w") as f:
    f.write(cannot_string)

In [19]:
print(cannot_string)

<?xml version="1.0" encoding="UTF-8"?>
<aiml version="2.0">

  <category>
    <pattern>$ALEXA *</pattern>
    <template>
      <srai><star/></srai>
    </template>
  </category>

  <category>
    <pattern>$AMAZON *</pattern>
    <template>
      <srai><star/></srai>
    </template>
  </category>

  <category>
    <pattern>$ECHO *</pattern>
    <template>
      <srai><star/></srai>
    </template>
  </category>

  <category>
    <pattern>$COMPUTER *</pattern>
    <template>
      <srai><star/></srai>
    </template>
  </category>


  <category>
    <pattern>PLEASE  *</pattern>
    <template>
      <srai><star/></srai>
    </template>
  </category>

  <category>
    <pattern>COULD YOU *</pattern>
    <template>
      <srai><star/></srai>
    </template>
  </category>

  <category>
    <pattern>CAN YOU *</pattern>
    <template>
      <srai><star/></srai>
    </template>
  </category>
  <category>
    <pattern>WHEN # MEETING #</pattern>
    <template>
      <srai>CAN YOU DO ALEXA THINGS</